In [149]:
import requests
import json
import pandas as pd

In [150]:
pd.options.display.max_columns = None

### Step 1. Prepare the plumbing

In [151]:
resource = 'https://xxxx.crm6.dynamics.com'
clientId = '00000000000000000000000000000000'
tenantId = '11111111111111111111111111111111'
grantType = 'client_credentials'
clientSecret = '22222222222222222222222222222222'
tokenendpoint = 'https://login.microsoftonline.com/11111111111111111111111111111111/oauth2/token'

In [152]:
base_url = 'https://orgfc831d9e.crm6.dynamics.com/api/data/v9.2'

In [153]:
audits_url = '/audits'

In [154]:
base_url + audits_url

'https://orgfc831d9e.crm6.dynamics.com/api/data/v9.2/audits'

In [155]:
tokenpost = {
    'client_id':clientId,
    'tenant_id': tenantId,
    'resource':resource,
    'client_secret': clientSecret,
    'grant_type': grantType,
}

In [156]:
tokenres = requests.post(tokenendpoint, data=tokenpost)

In [157]:
accesstoken = ''

In [158]:
try:
    accesstoken = tokenres.json()['access_token']
except(KeyError):
    #handle any missing key errors
    print('Could not get access token')

In [159]:
request_header = {
    'Authorization': 'Bearer ' + accesstoken,
    'OData-MaxVersion': '4.0',
    'OData-Version': '4.0',
    'Accept': 'application/json',
    'Content-Type': 'application/json; charset=utf-8',
    'Prefer': 'odata.maxpagesize=500',
    'Prefer': 'odata.include-annotations=OData.Community.Display.V1.FormattedValue'
    }

In [160]:
def get(endpoint, query = '', df = None):
    data = requests.get(endpoint + query, headers=request_header).json()
    if df is None:
        df = pd.DataFrame()
    if data is not None:
        if 'value' in data:
            df = df.append(pd.DataFrame.from_records(data['value']))
        if '@odata.nextLink' in data:
            query = urlparse(data['@odata.nextLink']).query
        return df

### Step 2. Get the audit log table for 'Project'

#### 2.1 Get audit records

In [161]:
project_logs = get(endpoint=base_url + audits_url, query="?$filter=objecttypecode eq 'bi_project'")

In [162]:
project_logs.head()

,_objectid_value@OData.Community.Display.V1.FormattedValue,_objectid_value,_userid_value@OData.Community.Display.V1.FormattedValue,_userid_value,operation@OData.Community.Display.V1.FormattedValue,operation,createdon@OData.Community.Display.V1.FormattedValue,createdon,auditid,attributemask,action@OData.Community.Display.V1.FormattedValue,action,objecttypecode@OData.Community.Display.V1.FormattedValue,objecttypecode,transactionid,_regardingobjectid_value,useradditionalinfo,_callinguserid_value
0,Riskman,0684bc6d-b158-ec11-8f8f-00224811ae90,Robin Du Admin,d6ec6dbc-be53-ec11-8f8e-00224811ce87,Update,2,2/10/2022 1:42 AM,2022-02-10T01:42:33Z,4476a5b6-128a-ec11-a507-dc98406867db,64,Update,2,Project,bi_project,991396b6-128a-ec11-93b0-000d3ad048c8,None,None,None
1,Robin's Test for conditional navigation,f37abab8-238d-ec11-b400-002248923fa2,Suan Chuah Admin,825d8b96-4543-ec11-8c62-0022481139a3,Update,2,2/15/2022 6:46 AM,2022-02-15T06:46:56Z,74d3b210-2b8e-ec11-a507-dc98406867db,66,Update,2,Project,bi_project,0796b110-2b8e-ec11-b400-002248923c72,None,None,None
2,Project Framework,390732e6-b35b-ec11-8f8f-000d3ad20434,Aditya Ambale Admin,64a73127-c5ff-eb11-94ef-0022481063de,Update,2,3/7/2022 10:39 AM,2022-03-07T10:39:16Z,f44444d3-029e-ec11-a22a-dc98400b1f42,66,Update,2,Project,bi_project,c8fbfbcf-029e-ec11-b400-0022489307fa,None,None,None
3,Capital Building 1,1e187487-4d59-ec11-8f8f-00224811ae90,Aditya Ambale Admin,64a73127-c5ff-eb11-94ef-0022481063de,Update,2,3/17/2022 11:00 PM,2022-03-17T23:00:02Z,902cccf5-45a6-ec11-a22a-dc98400b1f42,"71,73",Update,2,Project,bi_project,cdf196f9-45a6-ec11-9840-00224893729b,None,None,None
4,Test Adi - flow,901c8421-76b1-ec11-983f-0022489363a9,Aditya Ambale Admin,64a73127-c5ff-eb11-94ef-0022481063de,Create,1,4/1/2022 4:42 AM,2022-04-01T04:42:27Z,77362721-76b1-ec11-997e-a04a5ea2c656,"41,75,34,27,49,39,73,43,45,16,40,67,71,25",Create,1,Project,bi_project,951c8421-76b1-ec11-983f-0022489363a9,None,None,None


#### 2.2 Inteperate Attribute Mask

In [163]:
attribute_url = "https://orgfc831d9e.crm6.dynamics.com/api/data/v9.2/EntityDefinitions(LogicalName='bi_project')/Attributes"

In [164]:
attributes = get(endpoint=attribute_url)

In [165]:
project_logs['attributemask'] = project_logs['attributemask'].astype(str)

In [166]:
attribute2monitor = ['bi_fundedbudget', 'bi_forecast', 'bi_budgetspent', 'bi_budget']

In [167]:
attributeMask = attributes.loc[attributes['LogicalName'].isin(attribute2monitor), 'ColumnNumber'].tolist()

In [168]:
attributeMask = [str(x) for x in attributeMask]

In [169]:
attributeMask

['58', '41', '40', '45']

In [170]:
project_logs['attributemask'] = project_logs['attributemask'].str.split(',')

In [171]:
def attributeFilter(row):
    for item in attributeMask:
        if item in row['attributemask']:
            return True
        else:
            return False

In [172]:
mask = project_logs.apply(attributeFilter, axis=1)

In [173]:
project_logs.shape

(109, 18)

In [174]:
project_logs = project_logs[mask]

In [175]:
project_logs.head(2)

,_objectid_value@OData.Community.Display.V1.FormattedValue,_objectid_value,_userid_value@OData.Community.Display.V1.FormattedValue,_userid_value,operation@OData.Community.Display.V1.FormattedValue,operation,createdon@OData.Community.Display.V1.FormattedValue,createdon,auditid,attributemask,action@OData.Community.Display.V1.FormattedValue,action,objecttypecode@OData.Community.Display.V1.FormattedValue,objecttypecode,transactionid,_regardingobjectid_value,useradditionalinfo,_callinguserid_value
12,Robin's 3rd test,4029d3d8-b484-ec11-8d21-00224892953a,Robin Du Admin,d6ec6dbc-be53-ec11-8f8e-00224811ce87,Create,1,2/3/2022 5:48 AM,2022-02-03T05:48:03Z,dcdbb9d3-b484-ec11-a507-dc98406867db,"[45, 39, 41, 36, 49, 34, 27, 66, 58, 43, 16, 4...",Create,1,Project,bi_project,5e29d3d8-b484-ec11-8d21-00224892953a,None,None,None
14,NaN,5f12f99f-73b1-ec11-983f-0022489363a9,Aditya Ambale Admin,64a73127-c5ff-eb11-94ef-0022481063de,Delete,3,4/1/2022 4:41 AM,2022-04-01T04:41:29Z,d5d73dfa-75b1-ec11-997e-a04a5ea2c656,"[39, 46, 25, 27, 62, 66, 41, 51, 30, 34, 67, 7...",Delete,3,Project,bi_project,47ad66f9-75b1-ec11-983f-0022489363a9,None,None,None


In [176]:
project_logs.shape

(26, 18)

In [177]:
project_logs['createdon@OData.Community.Display.V1.FormattedValue'] = pd.to_datetime(project_logs['createdon@OData.Community.Display.V1.FormattedValue'])

In [178]:
project_logs.sort_values(by='createdon@OData.Community.Display.V1.FormattedValue', ascending=False).head(2)

,_objectid_value@OData.Community.Display.V1.FormattedValue,_objectid_value,_userid_value@OData.Community.Display.V1.FormattedValue,_userid_value,operation@OData.Community.Display.V1.FormattedValue,operation,createdon@OData.Community.Display.V1.FormattedValue,createdon,auditid,attributemask,action@OData.Community.Display.V1.FormattedValue,action,objecttypecode@OData.Community.Display.V1.FormattedValue,objecttypecode,transactionid,_regardingobjectid_value,useradditionalinfo,_callinguserid_value
27,Test Adi - flow,901c8421-76b1-ec11-983f-0022489363a9,Robin Du Admin,d6ec6dbc-be53-ec11-8f8e-00224811ce87,Update,2,2022-04-04 05:42:00,2022-04-04T05:42:57Z,55c1e513-dab3-ec11-997e-a04a5ea2c656,"[41, 58, 45, 40]",Update,2,Project,bi_project,e97a4713-dab3-ec11-983f-002248941db2,None,None,None
26,NaN,07cec962-7db1-ec11-983f-000d3acb6fc2,Aditya Ambale Admin,64a73127-c5ff-eb11-94ef-0022481063de,Delete,3,2022-04-01 05:53:00,2022-04-01T05:53:49Z,d29a6517-80b1-ec11-997e-a04a5ea2c656,"[39, 46, 25, 27, 62, 66, 41, 51, 30, 34, 67, 7...",Delete,3,Project,bi_project,b4e3d016-80b1-ec11-983f-0022489363a9,None,None,None


## Step 3. Retrieve 'Audit Details'

In [179]:
sample = 'https://orgfc831d9e.crm6.dynamics.com/api/data/v9.2/audits(4476a5b6-128a-ec11-a507-dc98406867db)/Microsoft.Dynamics.CRM.RetrieveAuditDetails()'

In [180]:
sample2 = 'https://orgfc831d9e.crm6.dynamics.com/api/data/v9.2/RetrieveRecordChangeHistory(Target=@p1,PagingInfo=@p2)?@p1={%27@odata.id%27:%27bi_projects(0684bc6d-b158-ec11-8f8f-00224811ae90)%27}&@p2={%27Count%27:10,%27PageNumber%27:1,%27ReturnTotalRecordCount%27:true}'

In [202]:
first_half_url = 'https://orgfc831d9e.crm6.dynamics.com/api/data/v9.2/audits('
last_half_url = ')/Microsoft.Dynamics.CRM.RetrieveAuditDetails()'

In [203]:
row = next(project_logs.iterrows())[1]

In [204]:
row

_objectid_value@OData.Community.Display.V1.FormattedValue                                     Robin's 3rd test
_objectid_value                                                           4029d3d8-b484-ec11-8d21-00224892953a
_userid_value@OData.Community.Display.V1.FormattedValue                                         Robin Du Admin
_userid_value                                                             d6ec6dbc-be53-ec11-8f8e-00224811ce87
operation@OData.Community.Display.V1.FormattedValue                                                     Create
operation                                                                                                    1
createdon@OData.Community.Display.V1.FormattedValue                                        2022-02-03 05:48:00
createdon                                                                                 2022-02-03T05:48:03Z
auditid                                                                   dcdbb9d3-b484-ec11-a507-dc98406867db
a

In [248]:
def retrieveAuditDetail(audit_id):
    result = requests.get(url = first_half_url + audit_id + last_half_url, headers=request_header).json()
    updated_item = {}
    if result['AuditDetail']['NewValue'] is not None:
        for item in attribute2monitor:
            try:
                updated_item.update({item + '_newValue': result['AuditDetail']['NewValue'][item]})
                try:
                    updated_item.update({item + '_oldValue': result['AuditDetail']['OldValue'][item]})
                except:
                    updated_item.update({item + '_oldValue': None})
            except:
                pass

    return updated_item

In [253]:
records = []

In [262]:
for index, row in project_logs.iterrows():
    result = retrieveAuditDetail(row['auditid'])
    try:
        result.update({'ProjectName': row['_objectid_value@OData.Community.Display.V1.FormattedValue']})
    except:
        pass
    result.update({'ProjectId': row['_objectid_value']})
    try:
        result.update({'UserName': row['_userid_value@OData.Community.Display.V1.FormattedValue ']})
    except:
        pass
    result.update({'UserId': row['_userid_value']})
    result.update({'CreatedOn': row['createdon']})
    result.update({'Action': row['action@OData.Community.Display.V1.FormattedValue']})
    records.append(result)

In [264]:
df = pd.DataFrame.from_records(records)

In [266]:
df.head()

,bi_fundedbudget_newValue,bi_fundedbudget_oldValue,bi_forecast_newValue,bi_forecast_oldValue,bi_budgetspent_newValue,bi_budgetspent_oldValue,bi_budget_newValue,bi_budget_oldValue,ProjectId,UserId,CreatedOn,Action,ProjectName
0,111.0,NaN,111.0,NaN,111.0,NaN,111.0,NaN,4029d3d8-b484-ec11-8d21-00224892953a,d6ec6dbc-be53-ec11-8f8e-00224811ce87,2022-02-03T05:48:03Z,Create,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5f12f99f-73b1-ec11-983f-0022489363a9,64a73127-c5ff-eb11-94ef-0022481063de,2022-04-01T04:41:29Z,Delete,NaN
2,111.0,NaN,1111.0,NaN,1111.0,NaN,111111.0,NaN,113c1a93-c393-ec11-b400-00224892b779,c669fb53-3277-ec11-8d21-0022489225eb,2022-02-22T09:41:14Z,Create,NaN
3,2222.0,NaN,4444.0,0.0,3333.0,0.0,111111.0,0.0,c9ca1505-886e-ec11-8943-000d3a6b5be3,d6ec6dbc-be53-ec11-8f8e-00224811ce87,2022-02-10T01:44:00Z,Update,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1fd8c150-7bb1-ec11-983f-0022489363a9,64a73127-c5ff-eb11-94ef-0022481063de,2022-04-01T05:25:30Z,Delete,NaN
